# Ensemble
Dags att rösta!


## Importera nödvändiga paket.

In [ ]:
# Av eget intresse vill jag gärna veta hur lång tid olika saker tar.
import time
notebook_start = time.time()  
t0 = time.time()

# Skippa varningar
import warnings
warnings.filterwarnings('ignore')

# Boosta prestandan
from sklearnex import patch_sklearn
patch_sklearn()

# Paket för datahantering
import numpy as np
import pandas as pd
import os
import math

# Dataset och modeller
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split 

# Preprocessing/pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.kernel_approximation import Nystroem

# Dimensionsreducering
from sklearn.decomposition import PCA

# Feature-bearbetning
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

# Data augmentation och förbehandling - stabil version för SciPy
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter, map_coordinates

# Bildbehandling
from PIL import Image

# Modeller 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# För export av modellen/scalern för vidare användning i Streamlit-appen
import joblib
from joblib import Parallel, delayed

# Slutrapport
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

## Läs in MNISt-datasetet och splitta det. 

*//Best practice: splitta ut testsetet direkt//*

In [ ]:
t0 = time.time()

# Läs in alla MNIST-data
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"]              
y = mnist["target"].astype(np.uint8)

# Splitta (80/20) med stratifiering för jämna klassfördelningar
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# Normalisering (en riktigt bra grej för både PCA och SVC!)
# Genom att dela med 255.0 blir alla värden mellan 0 och 1
X_train = X_train / 255.0
X_test = X_test / 255.0

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print("-" * 40)
print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

HoG ska boosta accuracy!

In [ ]:
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def local_hog(img_flat):
            return hog(img_flat.reshape(28, 28), 
                       orientations=self.orientations, 
                       pixels_per_cell=self.pixels_per_cell, 
                       cells_per_block=self.cells_per_block,
                       visualize=False)
        
        return np.array([local_hog(x) for x in X])

Definitioner

In [ ]:
t0 = time.time()

def deskew(image):
    # Räta upp lutande siffror 
    img = image.reshape(28, 28)
    
    # Skapa koordinat-matriser (y för rader, x för kolumner)
    y, x = np.mgrid[:28, :28]
    
    # Hitta tyngdpunkten (Center of Mass) med ndimage
    mu = ndimage.center_of_mass(img)
    if np.isnan(mu).any(): # Om bilden är tom
        return img.flatten()
    
    # Beräkna centrala moments (mu11 = kovarians, mu02 = varians i y-led)
    mu11 = np.sum((x - mu[1]) * (y - mu[0]) * img)
    mu02 = np.sum((y - mu[0])**2 * img)
    
    # Om variansen är för liten lutar siffran inte eller är för tunn
    if abs(mu02) < 1e-2:
        return img.flatten()
    
    # Skew-faktorn (förskjutning av x per enhet y)
    skew = mu11 / mu02
    
    # Här rätas x upp genom subtrahering av skew * y
    # Matrisen blir [[1, 0], [skew, 1]] med SciPys omvända ordning.
    matrix = np.array([[1, 0], [skew, 1]])
    
    # Offset för att rotera/skeva kring bildens centrum (14, 14)
    center = np.array([14, 14])
    offset = center - np.dot(matrix, center)
    
    # Transformation
    img_deskewed = ndimage.affine_transform(img, matrix, offset=offset, order=1, mode='constant', cval=0)
    return img_deskewed.flatten()

def shift_image(image, dx, dy):
    return ndimage.shift(image.reshape(28, 28), [dy, dx], cval=0, mode="constant").flatten()

def rotate_image(image, angle):
    return ndimage.rotate(image.reshape(28, 28), angle, reshape=False, cval=0, mode="constant").flatten()

def zoom_image(image, factor):
    rescaled = ndimage.zoom(image.reshape(28, 28), factor)
    if factor > 1.0: # Zooma in (klipp)
        start = int((rescaled.shape[0] - 28) / 2)
        final = rescaled[start:start+28, start:start+28]
    else: # Zooma ut (padda)
        pad = int((28 - rescaled.shape[0]) / 2)
        final = np.pad(rescaled, ((pad, 28-rescaled.shape[0]-pad), (pad, 28-rescaled.shape[1]-pad)), mode='constant')
    return final.flatten()

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Preparera testsetet så att jämförelsen blir rättvis.

In [ ]:
t0 = time.time()

# Här rätas testbilderna upp för att matcha modellen
X_test_deskewed = np.array([deskew(img) for img in X_test])

X_test_deskewed = X_test_deskewed.astype('float32')

cell_time = time.time() - t0
print("-" * 40)
print(f">>> Sökningen klar på {cell_time:.1f} sekunder.")

In [ ]:
# LADDA EXPERTERNA ---

print("Hämtar experterna från C-disken...")
# Vi laddar hela de färdiga pipelinen-objekten
svc_expert = joblib.load('C:/mnist_data/mnist_svc_hog_max.joblib')
knn_expert = joblib.load('C:/mnist_data/mnist_knn_hog_max.joblib')
rf_expert  = joblib.load('C:/mnist_data/mnist_rf_hog_max.joblib')

print("Alla tre specialister är på plats!")

In [ ]:
# SAMLA IN RÖSTER OCH KONFIDENS ---
import time

t0 = time.time()
print(f"Hämtar röster och konfidens från alla experter på {len(X_test_deskewed)} testbilder...")

# 1. Beräkna sannolikheter (Proba)
# Dessa krävs för din Soft Voting Audit och själva juryns röstning
print("- SVC beräknar konfidens...")
proba_svc = svc_expert.predict_proba(X_test_deskewed)

print("- KNN beräknar konfidens...")
proba_knn = knn_expert.predict_proba(X_test_deskewed)

print("- Random Forest beräknar konfidens...")
proba_rf  = rf_expert.predict_proba(X_test_deskewed)

# 2. Skapa individuella prediktioner (Labels)
# Vi tar argmax på sannolikheterna för att se vad varje expert gissar på egen hand.
# Dessa variabler krävs för visualiseringen och för att se vem som gör fel.
y_pred_svc = np.argmax(proba_svc, axis=1)
y_pred_knn = np.argmax(proba_knn, axis=1)
y_pred_rf  = np.argmax(proba_rf, axis=1)

print("-" * 40)
print(f"KLART! All data är insamlad och redo för juryn. Tid: {time.time() - t0:.1f} sekunder.")

In [ ]:
# OPTIMERING AV JURY-VIKTER (SOFT VOTING) ---

def optimize_soft_weights(p_svc, p_knn, p_rf, y_true):
    best_acc = 0
    best_w = (0.5, 0.25, 0.25)
    
    # Vi testar ett brett spann (0.4 till 0.7) för din SVC-expert
    for w_svc in np.linspace(0.4, 0.7, 7):
        remaining = 1.0 - w_svc
        # Fördela resten mellan KNN och RF
        for w_knn in np.linspace(0.1, remaining - 0.1, 5):
            w_rf = remaining - w_knn
            
            # Beräkna viktad sannolikhet
            final_p = (p_svc * w_svc) + (p_knn * w_knn) + (p_rf * w_rf)
            preds = np.argmax(final_p, axis=1)
            
            acc = np.mean(preds == y_true)
            if acc > best_acc:
                best_acc = acc
                best_w = (w_svc, w_knn, w_rf)
                
    return best_acc, best_w

# 1. Kör optimeringen
best_acc, weights = optimize_soft_weights(proba_svc, proba_knn, proba_rf, y_test)
w_svc, w_knn, w_rf = weights

# 2. Skapa de variabler som rapporten behöver (VIKTIGT!)
ensemble_acc = best_acc 
num_errors = int(round((1 - ensemble_acc) * len(y_test)))
final_proba = (proba_svc * w_svc) + (proba_knn * w_knn) + (proba_rf * w_rf)
y_final_pred = np.argmax(final_proba, axis=1)

print(f"OPTIMAL VIKTNING: SVC={w_svc:.2f}, KNN={w_knn:.2f}, RF={w_rf:.2f}")
print(f"Slutgiltig Accuracy: {ensemble_acc:.5f} ({num_errors} fel)")

In [ ]:
# ENSEMBLE RESCUE ANALYSIS (VEM RÄDDADE VEM?) ---

# 1. Ta reda på vad SVC-experten röstade på egen hand
y_svc_pred = np.argmax(proba_svc, axis=1)

# 2. Identifiera "Räddningar" (SVC hade fel, men juryn röstade rätt)
# Detta händer ofta när RF eller KNN är mycket säkrare på sin sak
rescued_indices = np.where((y_svc_pred != y_test) & (y_final_pred == y_test))[0]

# 3. Identifiera "Haverier" (SVC hade rätt, men juryn drog ner den i diket)
sabotaged_indices = np.where((y_svc_pred == y_test) & (y_final_pred != y_test))[0]

# 4. Skriv ut resultatet
print("ANALYS AV JURY-ARBETET:")
print("-" * 30)
print(f"Bilder juryn RÄDDADE från SVC:   {len(rescued_indices)}")
print(f"Bilder juryn SABOTERADE:         {len(sabotaged_indices)}")
print(f"Netto-vinst för din nya ensemble: {len(rescued_indices) - len(sabotaged_indices)} bilder")
print("-" * 30)

# 5. Visualisera de 5 första räddningarna
if len(rescued_indices) > 0:
    plt.figure(figsize=(15, 4))
    # Vi visar max 5 bilder
    num_to_show = min(5, len(rescued_indices))
    for i in range(num_to_show):
        idx = rescued_indices[i]
        plt.subplot(1, num_to_show, i+1)
        plt.imshow(X_test_deskewed[idx].reshape(28, 28), cmap='gray')
        plt.title(f"Rätt svar: {y_test[idx]}\nSVC sa: {y_svc_pred[idx]}\nJury sa: {y_final_pred[idx]}", fontsize=10)
        plt.axis('off')
    plt.suptitle("Exempel på svåra bilder som juryn 'Räddade'", fontsize=14, y=1.05)
    plt.show()

In [ ]:
# VISUALISERING OCH SLUTRAPPORT ---
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 1. Beräkna accuracy för varje enskild expert i denna körning
acc_svc = np.mean(y_pred_svc == y_test)
acc_knn = np.mean(y_pred_knn == y_test)
acc_rf  = np.mean(y_pred_rf == y_test)

# 2. Skapa grafisk sammanställning
models = ['SVC (Geometri)', 'KNN (Likhet)', 'Random Forest', 'MASTER ENSEMBLE']
accuracies = [acc_svc, acc_knn, acc_rf, ensemble_acc]

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
bars = plt.bar(models, accuracies, color=['#3498db', '#2ecc71', '#e74c3c', '#f1c40f'])
plt.ylim(0.97, 1.0)
plt.title('Accuracy: Experter vs Jury', fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.001, f"{yval:.4f}", ha='center', fontweight='bold')

# 3. Visa Confusion Matrix för juryn
plt.subplot(1, 2, 2)
cm = confusion_matrix(y_test, y_final_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(ax=plt.gca(), cmap='Blues', colorbar=False)
plt.title('Juryns Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 4. REDOGÖRELSE FÖR PROCESSEN (Att ha med i rapporten)
print("\n" + "="*60)
print("SAMMANFATTNING AV MODELLBYGGET")
print("="*60)
print(f"1. Förbehandling:   Deskewing (upprätning) av alla bilder.")
print(f"2. Feature Engine:  FeatureUnion av PCA (global) och HOG (lokal geometri).")
print(f"3. Träningsdata:    625 570 rader (Augmentering + Hard Negative Mining).")
print(f"4. Jurysystem:      Hard Voting mellan SVC, KNN och Random Forest.")
print("-" * 60)
print(f"SLUTRESULTAT:       {ensemble_acc:.5f} Accuracy ({num_errors} fel av {len(y_test)})")
print("="*60)

# 5. DIVERGENS-ANALYS (Vad bråkade de om?)
disagreement_idx = np.where((y_pred_svc != y_pred_knn) | (y_pred_svc != y_pred_rf))[0]
print(f"\n>>> ANALYS: Juryn var oense om {len(disagreement_idx)} bilder.")

plt.figure(figsize=(15, 4))
for i, idx in enumerate(disagreement_idx[:5]):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_test_deskewed[idx].reshape(28, 28), cmap='gray')
    plt.title(f"Sann siffra: {y_test[idx]}\nSVC:{y_pred_svc[idx]} KNN:{y_pred_knn[idx]} RF:{y_pred_rf[idx]}", fontsize=9)
    plt.axis('off')
plt.suptitle('Exempel på bilder där experterna röstade olika', fontsize=16, fontweight='bold', y=1.05)
plt.show()